# Предсказание тегов с помощью линейных моделей

Попробуем предсказывать теги к постам с сайта [StackOverflow](https://stackoverflow.com)

### Предварительная обработка текста

Для работы с текстом нам понадобится лист стоп-слов. Импортируем необходимые библиотеки.

In [1]:
import nltk
from nltk.corpus import stopwords

В этой задаче предстоит иметь дело с набором данных заголовков постов из StackOverflow. Предоставлено разбиение на 3 датасета: *train*, *validation* и *test*. Все корпусы кроме *test* содержат названия постов и соответствующие теги (доступно 100 тегов). Корпус *test* не содержит ответов.

In [2]:
from ast import literal_eval
import pandas as pd
import numpy as np

Функция `literal_eval` из библиотеки `ast` позволяет поменять тип данных `str` на другой подходящий. Выглядит это так:

In [3]:
# Создадим лист, но запишем его как строку
string = "['a', 'b', 'c']"
print(string)
print(type(string))

['a', 'b', 'c']
<class 'str'>


In [4]:
# Преобразуем строку в лист
str_to_list = literal_eval(string)
print(str_to_list)
print(type(str_to_list))

['a', 'b', 'c']
<class 'list'>


`literal_eval` сама определяет тип данных и может работать со словарями:

In [5]:
new_string = "{'a': 1, 'b': 2, 'c': 3}"
print(type(new_string))
str_to_dict = literal_eval(new_string)
print(type(str_to_dict))

<class 'str'>
<class 'dict'>


Теперь создадим функцию для чтения данных

In [6]:
def read_data(filename):
    data = pd.read_csv(filename, sep='\t') # Разделителем выступает символ табуляции
    data['tags'] = data['tags'].apply(literal_eval) # Применим literal_eval для нашей зависимой переменной
    return data

In [7]:
train = read_data('train.tsv')
validation = read_data('validation.tsv')
test = pd.read_csv('test.tsv', sep='\t')
# Заметьте, что в test мы используем функцию read_csv из библиотеки pandas, поэтому нам нужно указать разделитель
# Созданную нами функцию read_data мы не можем применить, потому что test не содержит зависимую переменную

Взглянем на данные

In [8]:
train.head()

,title,tags
0,How to draw a stacked dotplot in R?,[r]
1,mysql select all records where a datetime fiel...,"[php, mysql]"
2,How to terminate windows phone 8.1 app,[c#]
3,get current time in a specific country via jquery,"[javascript, jquery]"
4,Configuring Tomcat to Use SSL,[java]


In [9]:
validation.head()

,title,tags
0,Why odbc_exec always fail?,"[php, sql]"
1,Access a base classes variable from within a c...,[javascript]
2,"Content-Type ""application/json"" not required i...","[ruby-on-rails, ruby]"
3,Sessions in Sinatra: Used to Pass Variable,"[ruby, session]"
4,"Getting error - type ""json"" does not exist - i...","[ruby-on-rails, ruby, json]"


In [10]:
test.head()

,title
0,Warning: mysql_query() expects parameter 2 to ...
1,get click coordinates from <input type='image'...
2,How to implement cloud storage for media asset...
3,What is catcomplete in jQuery's autocomplete p...
4,Error building Android app with Cordova 3.1 CLI


Столбец `title` содержит заголовки, а столбец `tags` теги (удивительно). Важно то, что столбец `tags` не имеет фиксированного количества тегов.

Для удобства преобразуем наши данные в формат `np.array`

In [11]:
X_train, y_train = train['title'].values, train['tags'].values
X_val, y_val = validation['title'].values, validation['tags'].values
X_test = test['title'].values

Выглядят они теперь вот так:

In [12]:
X_train[:5]

array(['How to draw a stacked dotplot in R?',
       'mysql select all records where a datetime field is less than a specified value',
       'How to terminate windows phone 8.1 app',
       'get current time in a specific country via jquery',
       'Configuring Tomcat to Use SSL'], dtype=object)

In [13]:
y_train[:5]

array([list(['r']), list(['php', 'mysql']), list(['c#']),
       list(['javascript', 'jquery']), list(['java'])], dtype=object)

Одна из распространённых проблем при работе с естественными данными заключается в том, что они неструктурированы. Например, если бы мы использовали данные *как есть* и извлекали из них токены, просто разделяя их пробелами, мы бы увидели, что существует много "странных" токенов, таких как *3.5*,  *flip* и т.д. Чтобы предотвратить это, необходимо как-то подготовить данные. Здесь мы напишем функцию для решения этой проблемы.

In [14]:
import re

In [15]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]') # Сохраняем паттерн, который будем использовать позже
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]') # Ещё один паттерн
STOPWORDS = set(stopwords.words('english')) # Сохраняем  подмножество стоп-слов

In [16]:
# Как работает set? Он возвращает множество уникальных элементов
list_ = [7, 1, 2, 1, 2, 2, 3, 4, 6]
print(set(list_))

{1, 2, 3, 4, 6, 7}


In [17]:
def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # Меняем регистр всех символом на нижний
    text = re.sub(REPLACE_BY_SPACE_RE, ' ', text) # Меняем символы из REPLACE_BY_SPACE_RE на пробел в объекте text
    text = re.sub(BAD_SYMBOLS_RE, '', text) # Удаляем символы из BAD_SYMBOLS_RE (заменяем на ничего, не на пробел)
    text = text.split(' ') # Разделяем текст там, где пробелы
    text = [w for w in text if not w in STOPWORDS] # Создаём лист из слов текста, которых нет в STOPWRODS
    text = [w for w in text if not w == ''] # Создаём лист из слов, которые не "пустота"
    text = ' '.join(text) # Склеиваем лист с помощью пробела в одну строку
    return text

Проверим работу нашей функции

In [18]:
def test_text_prepare():
    examples = ["SQL Server - any equivalent of Excel's CHOOSE function?",
                "How to free c++ memory vector<int> * arr?"]
    answers = ["sql server equivalent excels choose function", 
               "free c++ memory vectorint arr"]
    for ex, ans in zip(examples, answers):
        if text_prepare(ex) != ans:
            return "Wrong answer for the case: '%s'" % ex
    return 'Basic tests are passed.'

In [19]:
print(test_text_prepare())

Basic tests are passed.


Теперь, когда мы убедились, что наша функция работает, применим её к нашим данным (это необходимо сделать только для признаков, но не для тегов).

In [20]:
X_train = [text_prepare(x) for x in X_train]
X_val = [text_prepare(x) for x in X_val]
X_test = [text_prepare(x) for x in X_test]

Выглядят они примерно так:

In [21]:
X_train[:5]

['draw stacked dotplot r',
 'mysql select records datetime field less specified value',
 'terminate windows phone 81 app',
 'get current time specific country via jquery',
 'configuring tomcat use ssl']

Для каждого тега (отклика, ответа) и слова (данные с признаками) посчитаем, как часто они встречаются в *train* корпусе.

In [22]:
# Создаём 2 пустых словаря для подсчёта тегов и слов
tags_counts = {}
words_counts = {}

for line in y_train: # Проходимся по каждому листу
    for word in line: # Проходимся по каждому слову в листе
        tags_counts.setdefault(word, 0) # Выставляем значение 0 для каждого слова, если оно не встретилось раньше
        # Если встретилось, предыдущая строчка пропускается
        tags_counts[word] += 1 # Считаем встречаемость

for line in X_train: # Тоже самое для признаков
    for word in line.split(): # Но здесь строчку надо поделить, по умолчанию стоит пробел
        words_counts.setdefault(word, 0)
        words_counts[word] += 1

Теперь мы можем найти самые встречаемые слова, просто отсортировав словари по значению.

In [23]:
most_common_tags = sorted(tags_counts.items(), key=lambda x: x[1], reverse=True)[:3]
# key здесь говорит о том, что мы берём 1-й элемент в словаре, то есть значение, а не ключ
most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:3]
print(most_common_tags, '\n', most_common_words)

[('javascript', 19078), ('c#', 19077), ('java', 18661)] 
 [('using', 8278), ('php', 5614), ('java', 5501)]


### Трансформация текста в вектор

Как мы знаем, алгоритмы машинного обучения могут работать только с числовыми данынми, поэтому мы не можем просто так использовать текст для обучения модели. Существует много методов преобразовать текст в числа, здесь мы познакомимся с двумя из них.

#### Bag of words

Один из самых популярных методов — метод мешка слов (bag of words). Чтобы преобразовать текст нам необходимо:
1. Найти *N* самых популярных слов в *train* корпусе и пронумеровать их. У нас уже есть словарь с самыми популярными словами.
2. Для каждого слова в корпусе создать нулевой вектор размерностью равной *N*.
3. Для каждого текста в корпусе перебрать слова, находящиеся в словаре и увеличивать их значение на 1.

Например, у нас есть список самых популярных слов:

    ['hi', 'you', 'me', 'are']

Мы их нумеруем и создаём словарь:

    {'hi': 0, 'you': 1, 'me': 2, 'are': 3}

И у нас есть текст, который надо трансформировать

    'hi how are you'

Для текста мы создаём нулевой вектор

    [0, 0, 0, 0]
    
И итерируемся по всем словам, если слово есть в словаре, мы увеличиваем число в векторе:

    'hi':  [1, 0, 0, 0] # hi — в нашем словаре имеет номер 0, значит встаёт на нулевую позицию
    'how': [1, 0, 0, 0] # слово 'how' отсутствует в словаре, поэтому на второй итерации ничего не происходит
    'are': [1, 0, 0, 1] # are в словаре на 3-й позиции, поэтому у нас появляется единичка на третей позиции
    'you': [1, 1, 0, 1]

Важно помнить, что этот вектор создаётся на основе словаря и он будет иметь его длину (а не длину предложения, здесь длины совпали)

Итоговый вектор:

    [1, 1, 0, 1]
   
Реализуем описанный метод в функции *my_bag_of_words* с размеров словаря равным 5000. Для поиска самых популярных слов исопльзуем *train* датасет. После проверим нашу функцию в *test_my_bag_of_words*.

In [24]:
DICT_SIZE = 5000

# Повторяем поиск популярных слов, но ограничиваем размер до 5000
most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:DICT_SIZE]

# Создадим словарь типа {0: 'using', 1: 'php', 2: 'java', 3: 'file', 4: 'javascript'...}
INDEX_TO_WORDS = dict(enumerate([i[0] for i in  most_common_words]))

# Создадим такой же словарь, но поменяем местами номера и слова (теперь сначала слова, потом их номера)
# Словарь типа {'using': 0, 'php': 1, 'java': 2, 'file': 3, 'javascript': 4...}
WORDS_TO_INDEX = {v:k for k, v in INDEX_TO_WORDS.items()}

# Создадим лист, куда поместимс все наши слова (без нумерации)
ALL_WORDS = WORDS_TO_INDEX.keys()

In [25]:
most_common_words[:5]

[('using', 8278),
 ('php', 5614),
 ('java', 5501),
 ('file', 5055),
 ('javascript', 4746)]

In [26]:
def my_bag_of_words(text, words_to_index, dict_size):
    """
        text: a string
        dict_size: size of the dictionary
        
        return a vector which is a bag-of-words representation of 'text'
    """
    # Создаём нулевой вектор размером словаря
    result_vector = np.zeros(dict_size)

    # Для каждого слова в тексте
    for w in nltk.word_tokenize(text):
        # Для каждого слова и индекса в словаре WORDS_TO_INDEX (подаётся вторым)
        for word, i in words_to_index.items():
            # Проверяем, есть ли слово в нашем словаре и если есть на место этого слова в векторе добавляем единичку
            if word == w:                     
                result_vector[i] += 1 
    
    return result_vector

In [27]:
def test_my_bag_of_words():
    words_to_index = {'hi': 0, 'you': 1, 'me': 2, 'are': 3}
    examples = ['hi how are you']
    answers = [[1, 1, 0, 1]]
    for ex, ans in zip(examples, answers):
        if (my_bag_of_words(ex, words_to_index, 4) != ans).any():
            return "Wrong answer for the case: '%s'" % ex
    return 'Basic tests are passed.'

In [28]:
print(test_my_bag_of_words())

Basic tests are passed.


Теперь применим нашу функцию ко всем данным (это может занять несколько минут)

In [29]:
from scipy import sparse as sp_sparse

In [30]:
# Для каждого объекта в данных с признаками применим функцию преобразования в вектор, потом сохраним этот вектор в сжатом виде
# А после чего соберём эти векторы в одну матрицу (добавлением строки вниз)

X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])
X_test_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_test])
print('X_train shape ', X_train_mybag.shape)
print('X_val shape ', X_val_mybag.shape)
print('X_test shape ', X_test_mybag.shape)

X_train shape  (100000, 5000)
X_val shape  (30000, 5000)
X_test shape  (20000, 5000)


Существует много представлений матриц в сжатом виде, однако sklearn умеет работать именно с таким типом, поэтому мы используем csr_matrix. Результат:

In [31]:
print('Как выглядела строка:', X_train[11])
print('Как она выглядит сейчас:', X_train_mybag[11].toarray()[0])
print('Количество ненулевых элементов в векторе строки:', np.count_nonzero(X_train_mybag[10].toarray()[0]))

Как выглядела строка: ipad selecting text inside text input tap
Как она выглядит сейчас: [0. 0. 0. ... 0. 0. 0.]
Количество ненулевых элементов в векторе строки: 7


7 — потому что в строке дважды слово *text*, следовательно, на одной позиции вектор имеет число 2

#### TF-IDF

Второй метод несколько расширяет возможности мешка слов и использует суммарные частоты появления слов в документах. Также этот подход наказывает за слишком частые слова.

Реализуем функцию *tfidf_features* с помощью класса [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) из *scikit-learn*. Потом мы используем *train* корпус чтобы обучить векторайзер. Важно настроить аргументы, которые мы собираемся ему передать. Мы предполагаем отфильтровать слишком редкие слова (которые появляются менее, чем в 5 документах) и слишком частые слова (появляются в более чем 90% документах). Также используем биграмы наряду с униграмами.

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [33]:
def tfidf_features(X_train, X_val, X_test):
    """
        X_train, X_val, X_test — samples        
        return TF-IDF vectorized representation of each sample and vocabulary
    """
    # Создаём TF-IDF векторайзер с аргуемнтами, которые мы выбрали
    tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2))

    # Обучаем векторайзер на train данных и трансформируем их
    X_train = tfidf_vectorizer.fit_transform(X_train)
    
    # Трансформируем по обученному векторайзеру наборы val и test и возвращаем всё
    X_val = tfidf_vectorizer.transform(X_val)
    X_test = tfidf_vectorizer.transform(X_test)
    
    return X_train, X_val, X_test, tfidf_vectorizer.vocabulary_

После предварительной обработки важно посмотреть на результаты. С этим нужно быть очень осторожныи, так как именно от этого зависит качество модели. 

В этом случае проверим, находятся ли c++ или c# в нашем наборе, так как очевидно, они являются важными маркерами.

In [34]:
X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_val, X_test)

In [35]:
# Создадим обратный словарь, где ключом будет частота слов, а значением - сами слова
tfidf_reversed_vocab = {i:word for word, i in tfidf_vocab.items()}

In [36]:
print('X_train_tfidf', X_train_tfidf.shape) 
print('X_test_tfidf', X_test_tfidf.shape) 
print('X_val_tfidf', X_val_tfidf.shape)

X_train_tfidf (100000, 17778)
X_test_tfidf (20000, 17778)
X_val_tfidf (30000, 17778)


In [37]:
'c++' in tfidf_reversed_vocab.values()

False

In [38]:
'c#' in tfidf_reversed_vocab.values()

False

Как видим, теги *c++* и *c#* не находятся в нашем словаре. Что же произошло? Здесь следует вернуться к токенизации с помощью TfidfVectorizer. К счастью, мы можем повлиять на это. Давайте повторим токенизацию с помощью паттерна regexp *(\S+)*.

In [39]:
def tfidf_features(X_train, X_val, X_test):
    """
        X_train, X_val, X_test — samples        
        return TF-IDF vectorized representation of each sample and vocabulary
    """
    # Создаём TF-IDF векторайзер с аргуемнтами, которые мы выбрали
    tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2), token_pattern='(\S+)')

    # Обучаем векторайзер на train данных и трансформируем их
    X_train = tfidf_vectorizer.fit_transform(X_train)
    
    # Трансформируем по обученному векторайзеру наборы val и test и возвращаем всё
    X_val = tfidf_vectorizer.transform(X_val)
    X_test = tfidf_vectorizer.transform(X_test)
    
    return X_train, X_val, X_test, tfidf_vectorizer.vocabulary_

In [40]:
X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_val, X_test)
tfidf_reversed_vocab = {i:word for word, i in tfidf_vocab.items()}

print('X_train_tfidf', X_train_tfidf.shape) 
print('X_test_tfidf', X_test_tfidf.shape) 
print('X_val_tfidf', X_val_tfidf.shape)

X_train_tfidf (100000, 18300)
X_test_tfidf (20000, 18300)
X_val_tfidf (30000, 18300)


In [41]:
'c++' in tfidf_reversed_vocab.values()

True

In [42]:
'c#' in tfidf_reversed_vocab.values()

True

### Многоклассовая классификация

Как мы уже заметили, в этой задаче ответ может принимать несколько меток (тегов). Чтобы работать с такими предсказаниями, нам нужно закодировать все предсказания как 0 и 1 для тегов. Для этого существует [MultiLabelBinarizer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html) из библиотеки *sklearn*.

In [43]:
from sklearn.preprocessing import MultiLabelBinarizer

In [44]:
tags_counts

{'r': 1727,
 'php': 13907,
 'mysql': 3092,
 'c#': 19077,
 'javascript': 19078,
 'jquery': 7510,
 'java': 18661,
 'ruby-on-rails': 3344,
 'ruby': 2326,
 'ruby-on-rails-3': 692,
 'json': 2026,
 'spring': 1346,
 'spring-mvc': 618,
 'codeigniter': 786,
 'class': 509,
 'html': 4668,
 'ios': 3256,
 'c++': 6469,
 'eclipse': 992,
 'python': 8940,
 'list': 693,
 'objective-c': 4338,
 'swift': 1465,
 'xaml': 438,
 'asp.net': 3939,
 'wpf': 1289,
 'multithreading': 1118,
 'image': 672,
 'performance': 512,
 'twitter-bootstrap': 501,
 'linq': 964,
 'xml': 1347,
 'numpy': 502,
 'ajax': 1767,
 'django': 1835,
 'laravel': 525,
 'android': 2818,
 'rest': 456,
 'asp.net-mvc': 1244,
 'web-services': 633,
 'string': 1573,
 'excel': 443,
 'winforms': 1468,
 'arrays': 2277,
 'c': 3119,
 'sockets': 579,
 'osx': 490,
 'entity-framework': 649,
 'mongodb': 350,
 'opencv': 401,
 'xcode': 900,
 'uitableview': 460,
 'algorithm': 419,
 'python-2.7': 421,
 'angularjs': 1353,
 'dom': 400,
 'swing': 759,
 '.net': 3872

In [45]:
mlb = MultiLabelBinarizer(classes=sorted(tags_counts.keys())) # Создаём объект кодировщика с классами из tags_count (отклики)
y_train = mlb.fit_transform(y_train) # Применяем его к train и val, на test мы не обучаемся
y_val = mlb.fit_transform(y_val)

Теперь реализуем функцию *train_classifier* для обучения классификатора. В этой задаче мы предполагаем использовать One-vs-Rest подход, который включён в класс [OneVsRestClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html). При таком подходе обучается столько классификаторов, сколько у нас тегов (k). За базовый классификатор возьмём [LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). Это довольно простой метод, однако он достаточно хорошо справляется с задачами классификации текста.

In [46]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier

In [47]:
def train_classifier(X_train, y_train):
    """
      X_train, y_train — training data
      
      return: trained classifier
    """
    
    model = OneVsRestClassifier(LogisticRegression()).fit(X_train, y_train)

    return model  

Обучим классификатор для разных наборов — Bag of Words и TF-IDF.

In [48]:
classifier_mybag = train_classifier(X_train_mybag, y_train)
print('BOW done.')
classifier_tfidf = train_classifier(X_train_tfidf, y_train)
print('TFIDF done.')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


BOW done.


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


TFIDF done.


Теперь мы можем прогнозировать данные. Мы будем использовать два типа прогноза - метки и decision function, последнее даёт нам сырые предсказания чисел типа -0.348 или 0.225, метки основываются на этих предсказаниях и выдают 1, если число положительное и 0 в противном случае (если граница на 0).

In [49]:
y_val_predicted_labels_mybag = classifier_mybag.predict(X_val_mybag)
y_val_predicted_scores_mybag = classifier_mybag.decision_function(X_val_mybag)

y_val_predicted_labels_tfidf = classifier_tfidf.predict(X_val_tfidf)
y_val_predicted_scores_tfidf = classifier_tfidf.decision_function(X_val_tfidf)

Посмотрим на предсказания

In [50]:
# Предсказания TF-IDF
y_val_pred_inversed = mlb.inverse_transform(y_val_predicted_labels_tfidf) # Обратная трансформация (мы же закодировали метки)
y_val_pred_inversed[:10]

[(),
 (),
 ('json', 'ruby-on-rails'),
 (),
 ('ruby-on-rails',),
 (),
 (),
 ('python',),
 ('javascript', 'jquery'),
 ('hibernate', 'java')]

In [51]:
# Сначала наши предсказания BoW
y_val_pred_inversed = mlb.inverse_transform(y_val_predicted_labels_mybag)
y_val_pred_inversed[:10]

[(),
 (),
 ('ruby-on-rails',),
 ('ruby',),
 ('json', 'ruby-on-rails'),
 (),
 (),
 ('python',),
 ('javascript', 'jquery'),
 ('hibernate', 'java')]

In [52]:
# Реальные метки
y_val_inversed = mlb.inverse_transform(y_val)
y_val_inversed[:10]

[('php', 'sql'),
 ('javascript',),
 ('ruby', 'ruby-on-rails'),
 ('ruby', 'session'),
 ('json', 'ruby', 'ruby-on-rails'),
 ('c++', 'ios', 'iphone', 'xcode'),
 ('c#',),
 ('django', 'python'),
 ('html', 'javascript', 'jquery'),
 ('hibernate', 'java')]

Теперь сравним результатыт и посмотрим, помогает ли TF-IDF или стоит попоробовать регуляризацию в логистической регрессии. Для всех этих экспериментов нам необходимо установить метрики.

### Метрики

Используем несколько метрик:
 - [Accuracy](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)
 - [F1-score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

In [122]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer

Реализуем функцию *print_evaluation_scores*, которая вычисляет и выдаёт:
 - *accuracy*
 - *F1-score macro/micro/weighted*
 - *Precision macro/micro/weighted*

In [89]:
def print_evaluation_scores(y_val, predicted):
    
    accuracy = accuracy_score(y_val, predicted)
    f1 = f1_score(y_val, predicted, average='weighted')
    precision = average_precision_score(y_val, predicted, average='weighted')
    recall = recall_score(y_val, predicted, average='weighted')
    print('Accuracy:', round(accuracy, 3))
    print('F1 score:', round(f1, 3))
    print('Precision:', round(precision, 3))
    print('Recall:', round(recall, 3))

In [90]:
print('Bag of words')
print_evaluation_scores(y_val, y_val_predicted_labels_mybag)
print()
print('TF-IDF')
print_evaluation_scores(y_val, y_val_predicted_labels_tfidf)

Bag of words
Accuracy: 0.349
F1 score: 0.65
Precision: 0.5
Recall: 0.573

TF-IDF
Accuracy: 0.334
F1 score: 0.614
Precision: 0.485
Recall: 0.501


Как видим, BoW в целом справился лучше с классификацией (хотя обе модели далеки от идеала).

### MultilabelClassification
После выбора метрики, можно поэкспериментировать с классифиактором. Будем использовать взвешенную F1-меру.

In [101]:
# Сначала попробуем поменять параметры в TF-IDF
def tfidf_features(X_train, X_val, X_test):
    tfidf_vectorizer = TfidfVectorizer(min_df=30, max_df=0.8, ngram_range=(1,2), token_pattern='(\S+)')
    X_train = tfidf_vectorizer.fit_transform(X_train)
    X_val = tfidf_vectorizer.transform(X_val)
    X_test = tfidf_vectorizer.transform(X_test)
    
    return X_train, X_val, X_test, tfidf_vectorizer.vocabulary_

X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_val, X_test)
tfidf_reversed_vocab = {i:word for word, i in tfidf_vocab.items()}

print('X_train_tfidf', X_train_tfidf.shape) 
print('X_test_tfidf', X_test_tfidf.shape) 
print('X_val_tfidf', X_val_tfidf.shape)

classifier_tfidf = train_classifier(X_train_tfidf, y_train)

y_val_predicted_labels_tfidf = classifier_tfidf.predict(X_val_tfidf)
y_val_predicted_scores_tfidf = classifier_tfidf.decision_function(X_val_tfidf)

# Предсказания TF-IDF
y_val_pred_inversed = mlb.inverse_transform(y_val_predicted_labels_tfidf) # Обратная трансформация (мы же закодировали метки)
y_val_pred_inversed[:10]

print_evaluation_scores(y_val, y_val_predicted_labels_tfidf)

Accuracy: 0.351
F1 score: 0.639
Precision: 0.497
Recall: 0.543


Результаты стали лучше, но это не сильно помогло